# Data Preprocessing
1. Import Libraries and download data
2. Scale training and test data using training data parameters to ensure no data leakage
3. Creates Sequences for training
4. Utilise Encoder-Decoder Network to reduce dimensionality of data to reduce overfitting

In [ ]:
random_state = 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Layer, Dropout, BatchNormalization, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import AdamW, SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MAE

In [ ]:
# Read dataset and set index to datetime for plotting purposes
data = pd.read_csv("/kaggle/input/eth-all/ETH_return2.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])
data.set_index("timestamp", inplace=True)


# Scale data using train parameters to ensure no data leakage
split = int(len(data) * 0.8)
train, test = data[:split], data[split:]
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)
scaled_data = np.vstack((train_scaled, test_scaled))



# Create Sequences
column_index = data.columns.get_loc('close')
predict_value = 24
SEQ_LENGTH = 24
def create_sequences(data, seq_length):
    X, Y = [], []
    for i in range(len(data) - seq_length - predict_value):  
        x = data[i:i + seq_length]
        y = data[i + seq_length + predict_value, column_index]  
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)
X, y = create_sequences(scaled_data, SEQ_LENGTH)


split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Auto-Encoder

In [ ]:
# Encoder - Decoder
dropout_rate = 0.2
activation = "tanh"
LSTM_units = 350
learning_rate = 0.1

input_layer = Input(shape=(X.shape[1], X.shape[2]))
encoded = LSTM(LSTM_units, activation=activation, return_sequences=True)(input_layer)
encoded = LSTM(4, activation=activation, return_sequences=True)(encoded)
decoded = LSTM(LSTM_units, return_sequences=True, activation=activation)(encoded)
decoded = Dropout(dropout_rate)(decoded)
decoded = TimeDistributed(Dense(X.shape[2]))(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=SGD(learning_rate=learning_rate, momentum=0.5), loss=MAE)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = autoencoder.fit(X, X, epochs=100, batch_size=128, validation_split=0.2, callbacks = [early_stopping])

encoder = Model(input_layer, encoded)
encoded_data = encoder.predict(X)


# Testing performance of encoder
reconstructed_sequences = autoencoder.predict(X)
mse = np.mean(np.power(X - reconstructed_sequences, 2), axis=(1, 2))
print(f'Mean Squared Error: {np.mean(mse)}')

# Forecasting Model

In [ ]:
split = int(len(encoded_data) * 0.8)
encoded_X_train, encoded_X_test = encoded_data[:split], encoded_data[split:]

lstm_units = 12
dropout_rate = 0.1
dense_units = 128
learning_rate = 0.001
weight_decay = 0.01
regularizer = 0.00001
num_layers = 2
activation = 'tanh'

input_seq = Input(shape=(SEQ_LENGTH, encoded_X_train.shape[2]))
for _ in range(num_layers):
    x = Bidirectional(LSTM(lstm_units, return_sequences=True, kernel_regularizer=l2(weight_decay), recurrent_regularizer=l2(regularizer)))(input_seq)
    x = Dropout(dropout_rate)(x)

x = Bidirectional(LSTM(lstm_units, return_sequences=False, kernel_regularizer=l2(weight_decay), recurrent_regularizer=l2(regularizer)))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x) 
x = Dropout(dropout_rate)(x)
output = Dense(1)(x)

model = Model(inputs=input_seq, outputs=output)
model.compile(optimizer=SGD(learning_rate=learning_rate, momentum=0.9), loss=MAE)


early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
history = model.fit(encoded_X_train, y_train, epochs=100, validation_split = 0.2, batch_size=128, callbacks = [early_stopping])
#model.save('Predict_24hr_model2.h5')

# Evaluation of Predictions

In [ ]:
#model = load_model("/kaggle/working/Predict_24hr.h5")

test_predictions = model.predict(encoded_X_test)

# This code is here to reset the datasets when running code multple times in one session
X, y = create_sequences(scaled_data, SEQ_LENGTH)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Inverse Scale Values
y_test_partial_reshaped = y_test.reshape(-1, 1)
y_test_reshaped = np.repeat(y_test_partial_reshaped, X_train.shape[2], axis=1)
preds_reshaped = np.repeat(test_predictions, X_train.shape[2], axis=1)
y_pred = scaler.inverse_transform(preds_reshaped)[:,column_index]
y_test = scaler.inverse_transform(y_test_reshaped)[:,column_index]


# Plot Preds vs Targets
plt.plot(y_test, label='Test Actual', color='orange')
plt.plot(y_pred, label='Test Predicted', color='red')
plt.xlabel('Time (Hours)')
plt.ylabel('Price')
plt.title('Actual vs Predicted Values')
plt.legend()
plt.show()


# Metrics
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
test_mse = mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mape = MAPE(y_test, y_pred)
print(f"Test MSE: {test_mse}")
print(f"Test MAE: {test_mae}")
print(f"Test MAPE: {test_mape}")

**Comparing predicted price change to actual price change in 24 hours**

In [ ]:
hours = 72

predicted_change = (y_pred[hours:] / y_test[:-hours]) - 1
actual_change = (y_test[hours:] / y_test[:-hours]) - 1

fig, ax = plt.subplots(1, 2, figsize=(8, 4)) 
ax[0].hist(predicted_change, bins=50)
ax[0].set_title(f"Predicted Price Change in {hours} hours (%)")
ax[0].set_xlabel("% change")

ax[1].hist(actual_change, bins=50)
ax[1].set_title(f"Actual Price Change in {hours} hours (%)")
ax[1].set_xlabel("% change")

plt.tight_layout()
plt.show()

**Identifying Performance in prediciting price direction**

In [ ]:
# Transforming Preds and Targets to % difference
predicted_direction = []
actual_direction = []

# Transforming preds and targets to 1 if upwards move and 0 if downwards move
for i in range(0,len(y_test)-predict_value,predict_value):
    if y_test[i+predict_value] >= y_test[i]:
        predicted_direction.append(1)
    else:
        predicted_direction.append(0)
        
for i in range(0,len(y_pred)-predict_value,predict_value):
    if y_pred[i+predict_value] >= y_test[i]:
        actual_direction.append(1)
    else:
        actual_direction.append(0)

# Plotting Confusion Matrix
matrix = metrics.confusion_matrix(actual_direction, predicted_direction)
cm_display = metrics.ConfusionMatrixDisplay(matrix, display_labels = ["down", "up"])
cm_display.plot()
plt.title("Prediction of Upward and Downward Price Movements")
plt.show()

**VaR of Returns (Historical Estimation)**

In [ ]:
confidence_level = 0.975
for x in range(24,97,24):
    pct_change = y_test[x:] / y_test[:-x]
    var_hist = 1 - np.percentile(pct_change, (1 - confidence_level) * 100)   
    print(f"{confidence_level * 100}% confident price will not fall more than {var_hist:.2%} in {x} hours")
print(" ")
confidence_level = 0.95
for x in range(24,97,24):
    pct_change = y_test[x:] / y_test[:-x]
    var_hist = 1 - np.percentile(pct_change, (1 - confidence_level) * 100)   
    print(f"{confidence_level * 100}% confident price will not fall more than {var_hist:.2%} in {x} hours")

# Trading Simulation

In [ ]:
prices = y_test[:-24]
preds = y_pred[24:]
actual = y_test[24:]

initial_balance = 10000
slippage = 0.0015  # binance trading fee (0.1%) + margin loss (0.05%)
balance = initial_balance
holdings = 0
buy_price = 0
profits = []
portfolio_value = []
actions = []



position_size = 1
take_profit = 1.1
max_hold_time = 48
signal_strength = 1.06

for i in range(len(prices)):

    current_price = prices[i]
    predicted_price = preds[i]
    signal = predicted_price / current_price
    action = "HOLD"
    
    if signal > signal_strength and balance > 0:
        amount_to_invest = balance*position_size
        shares_to_buy = amount_to_invest / current_price
        transaction_cost = shares_to_buy * current_price * slippage
        actual_shares_bought = (amount_to_invest - transaction_cost) / current_price
        holdings += actual_shares_bought
        balance -= amount_to_invest
        buy_price = current_price
        action = 'BUY'   

    elif current_price > buy_price*take_profit and holdings > 0:
            transaction_cost = holdings * current_price * slippage
            balance += (holdings * current_price) - transaction_cost
            profits.append(holdings * (current_price - buy_price) - transaction_cost)
            holdings = 0
            action = "PROFIT SELL"
        
        
    if i % max_hold_time == 0 :
        if holdings > 0:
            transaction_cost = holdings * current_price * slippage
            balance += (holdings * current_price) - transaction_cost
            profits.append(holdings * (current_price - buy_price) - transaction_cost)
            holdings = 0
            action = "TIME SELL"
    

    actions.append(action)  
    portfolio_value.append(balance + (holdings * current_price))

portfolio_value = np.array(portfolio_value)
portfolio = (portfolio_value * 100 / portfolio_value[0]) - 100
strategy_profit = ((portfolio_value[-1] / initial_balance) * 100) - 100

buy_and_hold = (prices * (100 - slippage) / prices[0]) - 100
buy_and_hold_profit = ((prices[-1] / prices[0]) * 100-slippage) - 100 


print(f"Strategy Profit: {strategy_profit:.2f}%")
print(f"Buy and Hold Profit: {buy_and_hold_profit:.2f}%")

index = test.index[predict_value+10:]
plt.figure(figsize=(9, 3))
plt.plot(index, portfolio, label="Strategy")
plt.plot(index, buy_and_hold, label="Buy And Hold") 
plt.xlabel('Time')
plt.ylabel('Profit (%)')
plt.title('Strategy vs Buy and Hold')
plt.legend()
plt.show()

print(pd.Series(actions).value_counts())

# Strategy Evaluation

In [ ]:
!pip install -q pandas pandas_datareader
import pandas_datareader.data as web
from datetime import datetime

df = web.DataReader('GS1', 'fred', datetime(2023, 1, 23),datetime(2024,6,2))
rf_annual = np.mean(df)/100
print(f"Risk Free Rate: {rf_annual}")

In [ ]:
PL_returns = np.diff(np.log(np.array(portfolio_value)))
market_returns = np.diff(np.log(np.array(y_test)))

def calc_max_drawdown(values):
    balances_series = pd.Series(values)
    running_max = balances_series.cummax()
    drawdown = (balances_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return -max_drawdown*100

def calculate_metrics(values, rf):
    annual_return = np.mean(values)*365*24
    volatility = np.std(values)*np.sqrt(365*24)
    max_drawdown = np.max(np.maximum.accumulate(values) - values) / np.max(np.maximum.accumulate(values))
    sharpe_ratio = (annual_return - rf) / volatility
    return volatility, max_drawdown, sharpe_ratio, annual_return

trading_metrics = calculate_metrics(PL_returns,rf_annual)
buy_and_hold_metrics = calculate_metrics(market_returns,rf_annual)
trading_drawdown = calc_max_drawdown(portfolio_value)
buy_and_hold_drawdown = calc_max_drawdown(prices)

# Print metrics
print("Trading Strategy Metrics:")
print(f"Annual Return: {trading_metrics[3]*100:.2f}%")
print(f"Volatility: {trading_metrics[0]:.2f}")
print(f"Maximum Drawdown: {trading_drawdown:.2f}%")
print(f"Sharpe Ratio: {trading_metrics[2]:.2f}")

print("\nBuy and Hold Metrics:")
print(f"Annual Return: {buy_and_hold_metrics[3]*100:.2f}%")
print(f"Volatility: {buy_and_hold_metrics[0]:.2f}")
print(f"Maximum Drawdown: {buy_and_hold_drawdown:.2f}%")
print(f"Sharpe Ratio: {buy_and_hold_metrics[2]:.2f}")

# Modelling Prices in various market dynamics

In [ ]:
import scipy.stats as stats
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.optimize import minimize
from scipy.stats import norm

# Assuming 'data' is your DataFrame with a DatetimeIndex

log_returns = pd.Series(np.diff(np.log(data["close"])))
log_returns = log_returns[np.isfinite(log_returns)]
market_conditions = {
    'flat': log_returns.loc['2018-01-01 00:00:00':'2021-01-01 00:00:00'],
    'bull': log_returns.loc['2021-01-01 00:00:00':'2022-01-01 00:00:00'],
    'bear': log_returns.loc['2022-01-01 00:00:00':'2023-01-01 00:00:00'],
    'test': log_returns.loc['2023-01-01 00:00:00':]
}

def kou_pdf(x, mu, sigma, lamb, p, eta1, eta2, delta_t=1):
    no_jump_pdf = (1 - lamb * delta_t) * norm.pdf(x, mu * delta_t, sigma * np.sqrt(delta_t))
    upward_jump_pdf = (lamb * delta_t * p * eta1 * np.exp((sigma**2 * eta1**2 * delta_t) / 2) *
                       np.exp(-eta1 * (x - mu * delta_t)) *
                       norm.cdf((x - mu * delta_t - sigma**2 * eta1 * delta_t) / (sigma * np.sqrt(delta_t))))
    downward_jump_pdf = (lamb * delta_t * (1 - p) * eta2 * np.exp((sigma**2 * eta2**2 * delta_t) / 2) *
                         np.exp(eta2 * (x - mu * delta_t)) *
                         norm.cdf((-x + mu * delta_t + sigma**2 * eta2 * delta_t) / (sigma * np.sqrt(delta_t))))
    return no_jump_pdf + upward_jump_pdf + downward_jump_pdf

def kou_cdf(x, mu, sigma, lamb, p, eta1, eta2, delta_t=1):
    cdf_values = np.array([np.sum(kou_pdf(xi, mu, sigma, lamb, p, eta1, eta2, delta_t) * delta_t) for xi in x])
    return cdf_values

def kou_log_likelihood(params, data):
    mu, sigma, lamb, p, eta1, eta2, delta_t = params
    pdf_values = kou_pdf(data, mu, sigma, lamb, p, eta1, eta2, delta_t)
    return -np.sum(np.log(pdf_values + 1e-9))  # Add small constant to avoid log(0)

def fit_kou_model(log_returns):
    mu, sigma = np.mean(log_returns), np.std(log_returns)
    threshold = 3 * sigma
    jump_indices = np.where(np.abs(log_returns - mu) > threshold)[0]
    lamb = len(jump_indices) / len(log_returns)
    jump_sizes = log_returns[jump_indices] - mu

    positive_jumps = jump_sizes[jump_sizes > 0]
    negative_jumps = -jump_sizes[jump_sizes < 0]
    eta1 = 1 / np.mean(positive_jumps) if len(positive_jumps) > 0 else 1.0
    eta2 = 1 / np.mean(negative_jumps) if len(negative_jumps) > 0 else 1.0
    p = len(positive_jumps) / len(jump_sizes) if len(jump_sizes) > 0 else 0.5

    initial_params = [mu, sigma, lamb, p, eta1, eta2, 1]
    return minimize(kou_log_likelihood, initial_params, args=(log_returns,), method='Nelder-Mead').x

def fit_distributions(log_returns):
    distributions = {"normal": stats.norm, "t": stats.t, "gpd": stats.genpareto}
    return {name: (dist, dist.fit(log_returns)) for name, dist in distributions.items()}

def goodness_of_fit(data, distribution, params):
    cdf = lambda x: distribution.cdf(x, *params)
    return stats.kstest(data, cdf)

def evaluate_best_fit(log_returns, fitted_params, kou_params):
    gof_results = {name: goodness_of_fit(log_returns, dist, params) for name, (dist, params) in fitted_params.items()}
    gof_results["kou"] = stats.kstest(log_returns, lambda x: kou_cdf(x, *kou_params))
    return min(gof_results, key=lambda k: gof_results[k][0]), gof_results

def plot_best_fit(log_returns, best_fit, best_params, fitted_params, kou_params):
    x = np.linspace(min(log_returns), max(log_returns), 1000)
    if best_fit == "kou":
        pdf = kou_pdf(x, *kou_params)
    else:
        pdf = fitted_params[best_fit][0].pdf(x, *best_params)
    
    plt.figure(figsize=(5, 3))
    plt.hist(log_returns, bins=50, density=True, alpha=0.6, color='g')
    plt.plot(x, pdf, 'k', linewidth=2)
    plt.title(f"Best fit distribution: {best_fit}")
    plt.show()

for condition, prices in market_conditions.items():
    kou_params = fit_kou_model(log_returns)
    fitted_params = fit_distributions(log_returns)
    best_fit, gof_results = evaluate_best_fit(log_returns, fitted_params, kou_params)
    best_params = kou_params if best_fit == "kou" else fitted_params[best_fit][1]
    plot_best_fit(log_returns, best_fit, best_params, fitted_params, kou_params)
    print(f"Best fitting distribution for {condition}: {best_fit}")
    if best_fit == "kou":
        print(f"Kou model parameters for {condition}: {kou_params}")
    else:
        print(f"{best_fit} parameters for {condition}: {best_params}")
